<a href="https://colab.research.google.com/github/Shriansh16/NLP/blob/main/Twitter_sentiment_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd

In [28]:
!unzip /content/Twitter_Data.zip

Archive:  /content/Twitter_Data.zip
replace Twitter_Data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [29]:
df=pd.read_csv('/content/Twitter_Data.csv')

In [30]:
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [31]:
df.shape

(162980, 2)

In [32]:
df['category'].value_counts()

 1.0    72250
 0.0    55213
-1.0    35510
Name: category, dtype: int64

In [33]:
df.isnull().sum()

clean_text    4
category      7
dtype: int64

In [34]:
df.dropna(inplace=True)

In [35]:
df.reset_index(inplace=True)

In [36]:
df

,index,clean_text,category
0,0,when modi promised “minimum government maximum...,-1.0
1,1,talk all the nonsense and continue all the dra...,0.0
2,2,what did just say vote for modi welcome bjp t...,1.0
3,3,asking his supporters prefix chowkidar their n...,1.0
4,4,answer who among these the most powerful world...,1.0
...,...,...,...
162964,162975,why these 456 crores paid neerav modi not reco...,-1.0
162965,162976,dear rss terrorist payal gawar what about modi...,-1.0
162966,162977,did you cover her interaction forum where she ...,0.0
162967,162978,there big project came into india modi dream p...,0.0


In [37]:
import numpy as np
import re
from nltk.corpus import stopwords

In [38]:
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [40]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## DATA PREPROCESSING

In [41]:
df1=df[df['category']!=0]

In [42]:
df1.reset_index(drop=True,inplace=True)

In [43]:
df1.drop(['index'],axis=1,inplace=True)

<ipython-input-43-72bfce017bb1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.drop(['index'],axis=1,inplace=True)


In [44]:
df1

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,what did just say vote for modi welcome bjp t...,1.0
2,asking his supporters prefix chowkidar their n...,1.0
3,answer who among these the most powerful world...,1.0
4,with upcoming election india saga going import...,1.0
...,...,...
107753,engine growth modi unveils indias first 12000 ...,1.0
107754,modi promised 2014 lok sabha elections that be...,1.0
107755,why these 456 crores paid neerav modi not reco...,-1.0
107756,dear rss terrorist payal gawar what about modi...,-1.0


In [45]:
df1.replace({'category':{-1:0}},inplace=True)

<ipython-input-45-b3a8babb0c37>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.replace({'category':{-1:0}},inplace=True)


In [46]:
df1

,clean_text,category
0,when modi promised “minimum government maximum...,0.0
1,what did just say vote for modi welcome bjp t...,1.0
2,asking his supporters prefix chowkidar their n...,1.0
3,answer who among these the most powerful world...,1.0
4,with upcoming election india saga going import...,1.0
...,...,...
107753,engine growth modi unveils indias first 12000 ...,1.0
107754,modi promised 2014 lok sabha elections that be...,1.0
107755,why these 456 crores paid neerav modi not reco...,0.0
107756,dear rss terrorist payal gawar what about modi...,0.0


In [47]:
df1['category'].value_counts()

1.0    72249
0.0    35509
Name: category, dtype: int64

### stemming: reducing a word into its root word
### eg: acting,actor,actress --> act

In [48]:
port_stem=PorterStemmer()

In [49]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)    # remove everything that are not alphabet
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [50]:
df1['stemmed_tweets']=df1['clean_text'].apply(stemming)

<ipython-input-50-cc5a4bdc3cc3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['stemmed_tweets']=df1['clean_text'].apply(stemming)


In [51]:
df1

,clean_text,category,stemmed_tweets
0,when modi promised “minimum government maximum...,0.0,modi promis minimum govern maximum govern expe...
1,what did just say vote for modi welcome bjp t...,1.0,say vote modi welcom bjp told rahul main campa...
2,asking his supporters prefix chowkidar their n...,1.0,ask support prefix chowkidar name modi great s...
3,answer who among these the most powerful world...,1.0,answer among power world leader today trump pu...
4,with upcoming election india saga going import...,1.0,upcom elect india saga go import pair look cur...
...,...,...,...
107753,engine growth modi unveils indias first 12000 ...,1.0,engin growth modi unveil india first electr lo...
107754,modi promised 2014 lok sabha elections that be...,1.0,modi promis lok sabha elect best orop given so...
107755,why these 456 crores paid neerav modi not reco...,0.0,crore paid neerav modi recov congress leader h...
107756,dear rss terrorist payal gawar what about modi...,0.0,dear rss terrorist payal gawar modi kill plu m...


In [93]:
# SEPARATING DATA AND LABEL COLUMN
X=df1['stemmed_tweets'].values
y=df1['category'].values

In [94]:
## splitting the data to training data and testing data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=20)

In [95]:
## converting the textual data to numerical data
vectorizer=TfidfVectorizer()

In [96]:
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [97]:
X_train

<86206x57758 sparse matrix of type '<class 'numpy.float64'>'
	with 1297433 stored elements in Compressed Sparse Row format>

In [98]:
print(X_train)

  (0, 48500)	0.3803394631397305
  (0, 29315)	0.3208009341781091
  (0, 42186)	0.25760431010215973
  (0, 32657)	0.05176597286330122
  (0, 23614)	0.1263301119597884
  (0, 51979)	0.17702007221196986
  (0, 40624)	0.2123087596356915
  (0, 52183)	0.21635937630976654
  (0, 7268)	0.7377039816826528
  (1, 37384)	0.49902304766368505
  (1, 4682)	0.3435082971225462
  (1, 322)	0.4122213699375887
  (1, 55280)	0.19055165606441743
  (1, 19419)	0.20388740449808773
  (1, 55593)	0.20301007377915867
  (1, 56531)	0.29144139085230947
  (1, 51213)	0.26478337126439566
  (1, 51554)	0.2368542143619749
  (1, 27713)	0.2957197890217909
  (1, 27895)	0.20512472718812166
  (1, 32657)	0.057305394975740644
  (2, 19075)	0.44005772427351675
  (2, 32413)	0.21776610941486246
  (2, 2546)	0.2426854978495884
  (2, 41763)	0.3250931757838423
  :	:
  (86202, 19912)	0.09998424739950401
  (86202, 43797)	0.17272451813249523
  (86202, 8095)	0.1760829967190906
  (86202, 13338)	0.12720030189868325
  (86202, 7211)	0.21004427572792397
  

In [99]:
## model training using logistic regression

In [100]:
model=LogisticRegression(max_iter=1000)   # maximum no. of times the model will go through the data

In [101]:
model.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [102]:
## lets check the accuracy

In [103]:
predictions=model.predict(X_test)

In [104]:
print(accuracy_score(y_test,predictions))

0.883398292501856


In [105]:
print(y_test,predictions)

[0. 1. 1. ... 1. 1. 1.] [0. 1. 1. ... 1. 1. 0.]


In [127]:
yy="The Modi government has done an excellent job in terms of highway construction"

In [128]:
k=stemming(yy)

In [129]:
kk=[k]

In [130]:
kk

['modi govern done excel job term highway construct']

In [131]:
k2=vectorizer.transform(kk)

In [132]:
k2

<1x57758 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [133]:
model.predict(k2)

array([1.])

In [134]:
## SAVING THE MODEL

In [135]:
import pickle

In [136]:
filename='sentiment_prediction.pkl'
pickle.dump(model,open(filename,'wb'))

In [137]:
## loading the model
load_model=pickle.load(open('/content/sentiment_prediction.pkl','rb'))

In [138]:
load_model.predict(k2)

array([1.])